In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

from env import get_connection

In [2]:
def get_zillow():
    
    filename = 'zillow.csv'
    
    if os.path.isfile(filename):
        
        print('found data')
        
        return pd.read_csv(filename)
    
    else:
        
        print('retrieving data')
        
        query = '''
                SELECT bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, fips
                FROM properties_2017
                WHERE propertylandusetypeid = '261';
                '''
        
        url = get_connection('zillow')
        
        df = pd.read_sql(query, url)
        
        df.to_csv(filename, index = 0)
        
        return df

In [3]:
df = get_zillow()
df.head()

found data


,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0


In [4]:
df.shape

(2152863, 7)

In [5]:
df.fips.value_counts()

6037.0    1431811
6059.0     555077
6111.0     165975
Name: fips, dtype: int64

In [6]:
df.rename(columns={'bedroomcnt': 'bedrooms', 'bathroomcnt': 'bathrooms', 'calculatedfinishedsquarefeet': 'square_ft', 'taxvaluedollarcnt': 'tax_value', 'yearbuilt': 'year', 'taxamount': 'tax_amount'}, inplace=True)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2152863 entries, 0 to 2152862
Data columns (total 7 columns):
 #   Column      Dtype  
---  ------      -----  
 0   bedrooms    float64
 1   bathrooms   float64
 2   square_ft   float64
 3   tax_value   float64
 4   year        float64
 5   tax_amount  float64
 6   fips        float64
dtypes: float64(7)
memory usage: 115.0 MB


## year, bed, sqft should be integers. I have to cast the rest after I determine what to do with null values

In [8]:
#check for null values
df.isna().sum()

bedrooms        11
bathrooms       11
square_ft     8484
tax_value      493
year          9337
tax_amount    4442
fips             0
dtype: int64

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2152863 entries, 0 to 2152862
Data columns (total 7 columns):
 #   Column      Dtype  
---  ------      -----  
 0   bedrooms    float64
 1   bathrooms   float64
 2   square_ft   float64
 3   tax_value   float64
 4   year        float64
 5   tax_amount  float64
 6   fips        float64
dtypes: float64(7)
memory usage: 115.0 MB


## I will impute median year based of fip for the year null values

In [10]:
#what fips do null values in year belong to
df[df.year.isna()].fips.value_counts()

6037.0    3775
6111.0    2943
6059.0    2619
Name: fips, dtype: int64

## I need to impute median year built by fip for null values

In [11]:
#median year in 6037 fip
df[df.fips == 6037].year.median()

1953.0

In [12]:
#median year in 6111 fip
df[df.fips == 6111].year.median()

1973.0

In [13]:
#median year in 6059 fip
df[df.fips == 6059].year.median()

1970.0

In [14]:
# fill year nulls in 6037 fip with median year
df.loc[df['fips'] == 6037, 'year'] = df.loc[df['fips'] == 6037, 'year'].fillna(1953)

In [15]:
df.year[df.fips == 6037].isna().sum()

0

In [16]:
# fill year nulls in 6111 fip with median year
df.loc[df['fips'] == 6111, 'year'] = df.loc[df['fips'] == 6111, 'year'].fillna(1973)


In [17]:
df.year[df.fips == 6111].isna().sum()

0

In [18]:
# fill year nulls in 6059 fip with median year
df.loc[df['fips'] == 6059, 'year'] = df.loc[df['fips'] == 6059, 'year'].fillna(1970)

In [19]:
df.year[df.fips == 6059].isna().sum()

0

In [20]:
#check total nulls
df.isna().sum()

bedrooms        11
bathrooms       11
square_ft     8484
tax_value      493
year             0
tax_amount    4442
fips             0
dtype: int64

In [21]:
#cast to int
df.year = df.year.astype('int')

## For square ft nulls I will do a similar approach and determine what the median square ft is grouped by fip and fill nulls with median values

In [22]:
#what fips do null values in square ft belong to
df[df.square_ft.isna()].fips.value_counts()

6037.0    3495
6059.0    2717
6111.0    2272
Name: fips, dtype: int64

In [23]:
#median square ft in 6037 fip
df[df.fips == 6037].square_ft.median()

1535.0

In [24]:
#median square ft in 6059 fip
df[df.fips == 6059].square_ft.median()

1831.0

In [25]:
#median square ft in 6111 fip
df[df.fips == 6111].square_ft.median()

1811.0

In [26]:
#fill square ft nulls in 6037 fip with median square ft
df.loc[df['fips'] == 6037, 'square_ft'] = df.loc[df['fips'] == 6037, 'square_ft'].fillna(1535)

In [27]:
#fill square ft nulls in 6059 fip with median square ft
df.loc[df['fips'] == 6059, 'square_ft'] = df.loc[df['fips'] == 6059, 'square_ft'].fillna(1831)

In [28]:
#fill square ft nulls in 6111 fip with median square ft
df.loc[df['fips'] == 6111, 'square_ft'] = df.loc[df['fips'] == 6111, 'square_ft'].fillna(1811)

In [29]:
#make sure there are no more nulls
df[df.square_ft.isna()].fips.value_counts()

Series([], Name: fips, dtype: int64)

In [30]:
#check total nulls
df.isna().sum()

bedrooms        11
bathrooms       11
square_ft        0
tax_value      493
year             0
tax_amount    4442
fips             0
dtype: int64

In [31]:
#cast to int
df.square_ft = df.square_ft.astype('int')

## Investigate bedrooms and bathrooms

In [32]:
df[df.bedrooms == 0].count()

bedrooms      13187
bathrooms     13187
square_ft     13187
tax_value     12803
year          13187
tax_amount    11245
fips          13187
dtype: int64

## There are 13,187 properties with zero rooms. Why?

In [33]:
df[df.bathrooms == 0].count()

bedrooms      13027
bathrooms     13027
square_ft     13027
tax_value     12642
year          13027
tax_amount    11112
fips          13027
dtype: int64

## There are 13,027 properties with zero bathrooms. Why?

In [34]:
df[df.bedrooms.isna()].square_ft.value_counts()

1535    5
1831    2
1348    1
200     1
990     1
400     1
Name: square_ft, dtype: int64

## I will drop nulls and zeros with median bedrooms grouped by fip

In [35]:
# Drop rows with null values in the specified column
df.dropna(subset = ['bedrooms'], inplace = True)

In [36]:
df.dropna(subset = ['bathrooms'], inplace = True)

In [37]:
#check total nulls
df.isna().sum()

bedrooms         0
bathrooms        0
square_ft        0
tax_value      493
year             0
tax_amount    4433
fips             0
dtype: int64

## For now i will drop remainding nulls. 

In [38]:
df = df.dropna()

In [39]:
#check total nulls
df.isna().sum()

bedrooms      0
bathrooms     0
square_ft     0
tax_value     0
year          0
tax_amount    0
fips          0
dtype: int64

In [40]:
#cast to int
df.tax_value = df.tax_value.astype('int')

In [41]:
#cast to int
df.bedrooms = df.bedrooms.astype('int')

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2148388 entries, 3 to 2152862
Data columns (total 7 columns):
 #   Column      Dtype  
---  ------      -----  
 0   bedrooms    int64  
 1   bathrooms   float64
 2   square_ft   int64  
 3   tax_value   int64  
 4   year        int64  
 5   tax_amount  float64
 6   fips        float64
dtypes: float64(3), int64(4)
memory usage: 131.1 MB


## With more time I need to determine if tax value is correlated to bedrooms, year, square ft, and fip. If true I can potentially fill nulls with median tax value of properties with median bedrooms, year, square ft grouped by fip. Else I will drop

## With time I will leave tax amount null values and determine what to do with them when exploring for correlation with tax value on my train data set. I can potentially fill nulls with median tax value multiplied by average tax rate in corresponding fip

# I will rename fips to county and rename values to corresponding county name

In [43]:
df['fips'] = df['fips'].map({6037: 'Los Angeles', 6059: 'Orange', 6111: 'Ventura'})

In [44]:
df.rename(columns = {'fips': 'county'}, inplace = True)

In [45]:
df.county.value_counts()

Los Angeles    1427802
Orange          554773
Ventura         165813
Name: county, dtype: int64

In [46]:
df.head()

,bedrooms,bathrooms,square_ft,tax_value,year,tax_amount,county
3,0,0.0,1535,2108,1953,174.21,Los Angeles
4,4,2.0,3633,296425,2005,6941.39,Los Angeles
6,3,4.0,1620,847770,2011,10244.94,Los Angeles
7,3,2.0,2077,646760,1926,7924.68,Los Angeles
8,0,0.0,1535,6730242,1953,80348.13,Los Angeles


In [47]:
def clean_zillow():
    
    df = get_zillow()
    
    df = df.rename(columns={'bedroomcnt': 'bedrooms', 'bathroomcnt': 'bathrooms', 'calculatedfinishedsquarefeet': 'square_ft', 'taxvaluedollarcnt': 'tax_value', 'yearbuilt': 'year', 'taxamount': 'tax_amount'})
    
    # fill year nulls in fip with median year
    df.loc[df['fips'] == 6037, 'year'] = df.loc[df['fips'] == 6037, 'year'].fillna(1953)
    
    df.loc[df['fips'] == 6111, 'year'] = df.loc[df['fips'] == 6111, 'year'].fillna(1973)
    
    df.loc[df['fips'] == 6059, 'year'] = df.loc[df['fips'] == 6059, 'year'].fillna(1970)
    
    #fill square ft nulls in 6037 fip with median square ft
    df.loc[df['fips'] == 6037, 'square_ft'] = df.loc[df['fips'] == 6037, 'square_ft'].fillna(1535)
    
    df.loc[df['fips'] == 6059, 'square_ft'] = df.loc[df['fips'] == 6059, 'square_ft'].fillna(1831)
    
    df.loc[df['fips'] == 6111, 'square_ft'] = df.loc[df['fips'] == 6111, 'square_ft'].fillna(1811)
    
    #cast to int
    df.tax_value = df.tax_value.astype('int')
    
    df.year = df.year.astype('int')
    
    df.square_ft = df.square_ft.astype('int')
    
    df.bedrooms = df.bedrooms.astype('int')
    
    #drop remainding nulls
    df = df.dropna()
    
    #rename fips column and values
    df['fips'] = df['fips'].map({6037: 'Los Angeles', 6059: 'Orange', 6111: 'Ventura'})
    
    df = df.rename(columns = {'fips': 'county'})
    
    return df